In [18]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
result_dt = pd.read_excel('/content/drive/MyDrive/742 Financial Engineering Team 4/Week 7 /Data/result_dt.xlsx')
result_dt = result_dt.sort_values(by=0)

In [27]:
data1=pd.read_csv('/content/drive/MyDrive/742 Financial Engineering Team 4/Week 7 /Data/allcasdataincsvformat10000.csv',index_col = 0)
data1.head()

,Loan Identifier,Monthly Reporting Period,Channel,Seller Name,Servicer Name,Master Servicer,Original Interest Rate,Current Interest Rate,Original UPB,UPB at Issuance,...,Mortgage Margin,ARM Balloon Indicator,ARM Plan Number,Borrower Assistance Plan,High Loan to Value (HLTV) Refinance Option Indicator,Deal Name,Repurchase Make Whole Proceeds Flag,Alternative Delinquency Resolution,Alternative Delinquency Resolution Count,Total Deferral Amount
Reference Pool ID,,,,,,,,,,,,,,,,,,,,,
5128,95314561,112019,C,Other,Other,FANNIE MAE,4.500,4.500,315000,302502.49,...,NaN,NaN,NaN,NaN,N,CAS 2020 R02 G2,NaN,NaN,NaN,NaN
5128,95314562,112019,R,Other,Other,FANNIE MAE,5.125,5.125,133000,131811.27,...,NaN,NaN,NaN,NaN,N,CAS 2020 R02 G2,NaN,NaN,NaN,NaN
5128,95314563,112019,R,Other,Arvest Central Mortgage Company,FANNIE MAE,4.375,4.375,522000,505284.23,...,NaN,NaN,NaN,NaN,N,CAS 2020 R02 G2,NaN,NaN,NaN,NaN
5128,95314564,112019,B,"United Shore Financial Services, LLC d/b/a United",New Residential Mortgage LLC,FANNIE MAE,4.875,4.875,124000,122078.57,...,NaN,NaN,NaN,NaN,N,CAS 2020 R02 G2,NaN,NaN,NaN,NaN
5128,95314565,112019,R,Other,Other,FANNIE MAE,5.000,5.000,131000,129346.73,...,NaN,NaN,NaN,NaN,N,CAS 2020 R02 G2,NaN,NaN,NaN,NaN


In [28]:
#Projecting the UPB's for each loan for each of the 40 quarters
Monthly_Balance_Cas =[]
sumUPB=0
for index,row in data1.iterrows():
  sumUPB=sumUPB+ row['Original UPB']
  Loan_Amount = row['Original UPB']
  Interest_rate = row['Current Interest Rate']
    
  Loan_Term = 360
  R = 1+(Interest_rate)/(12*100)
  X = Loan_Amount * (R**Loan_Term)*(1-R)/(1-R**Loan_Term)

  Monthly_Interest = []
  Monthly_Balance = []

  for i in range(1, 123):
    Interest = Loan_Amount*(R-1)
    Loan_Amount = (Loan_Amount - (X-Interest))
    if (i%3 ==0) and (i != 1):
      Monthly_Interest.append(Interest)
      Monthly_Balance.append(Loan_Amount)
            

  Monthly_Balance_Cas.append(Monthly_Balance)

In [29]:
# Tranch Valuation by calculating the losses and payoffs for each of the loss scenarios
d1=100-3.92
d2=3.92-2.26
d3=2.26-1.73
d4=1.73-0.73
d5=0.73-0.00000000000000000000000000000000000000000000000000000001
loss_subordination=[0,0.73,1.73,2.26,3.92]
CDO_percentage=[d5,d4,d3,d2,d1]
referencepoolsize=sumUPB
tranchsize=[]
for i in CDO_percentage:
    tranchsize.append((referencepoolsize*i)/100)

In [30]:
Tranche = pd.DataFrame()
Tranche['Loss Subordination'] = loss_subordination
Tranche['CDO %'] = CDO_percentage
Tranche['Tranche size'] = tranchsize
Tranche

,Loss Subordination,CDO %,Tranche size
0,0.00,0.73,2.148003e+08
1,0.73,1.00,2.942470e+08
2,1.73,0.53,1.559509e+08
3,2.26,1.66,4.884501e+08
4,3.92,96.08,2.827126e+10


In [31]:
Tranch_Loss =[]*500
Tranch_Loss1=[]*500
Tranch_Loss2=[]*500
Tranch_Loss3=[]*500
Tranch_Loss4=[]*500
Tranch_Payoff = []
losses = [0]*500
payoff = [0]*500 
losses1 = [0]*500
payoff1 = [0]*500 
losses2=[0]*500
payoff2 = [0]*500 
losses3=[0]*500
payoff3 = [0]*500 
losses4=[0]*500
payoff4 = [0]*500 
#quarterly_rate = (((1.0254**(1/4))-1)*4)
for i in range(0,len(Tranche['Tranche size'])):
    
    payoffs =[]
    j=-1
    
    for el in result_dt[0]:
        j=j+1
        if(i==0):
            x = (min(Tranche['Tranche size'][i],(el*sumUPB/100)))
            losses[j]=x
            payoff[j]=Tranche['Tranche size'][i]-x
        elif(i==1):
            #print("hello")
            x=min(Tranche['Tranche size'][i],((el*sumUPB/100)-losses[j]))
            losses1[j]=x
            payoff1[j]=Tranche['Tranche size'][i]-x
            #print(x)
            #print(losses1[j])
        elif(i==2):
            x=min(Tranche['Tranche size'][i],((el*sumUPB/100)-losses[j]-losses1[j]))
            payoff2[j]=Tranche['Tranche size'][i]-x
            losses2[j]=x
        elif(i==3):
            x=min(Tranche['Tranche size'][i],((el*sumUPB/100)-losses[j]-losses1[j]-losses2[j]))
            payoff3[j]=Tranche['Tranche size'][i]-x
            losses3[j]=x
        else:
            x=min(Tranche['Tranche size'][i],((el*sumUPB/100)-losses[j]-losses1[j]-losses2[j]-losses3[j]))
            payoff4[j]=Tranche['Tranche size'][i]-x
            losses4[j]=x
        
        #payoffs.append(Tranche['Tranche size'][i]-x)
    
    if(i==0):
        Tranch_Loss.append(losses)
    elif(i==1):    
        Tranch_Loss1.append(losses1)
    elif(i==2):    
        Tranch_Loss2.append(losses2)
    elif(i==3):    
        Tranch_Loss3.append(losses3)
    else:
        Tranch_Loss4.append(losses4)
    
    #Tranch_Loss.append(losses)
    #Tranch_Payoff.append(payoffs)   

In [32]:
Price1=0
Price2=0
Price3=0
Price4=0
Price5=0
payoff=[i*0.002 for i in payoff]
payoff1=[i*0.002 for i in payoff1]
payoff2=[i*0.002 for i in payoff2]
payoff3=[i*0.002 for i in payoff3]
payoff4=[i*0.002 for i in payoff4]

In [33]:
for i in range(0,len(Tranche['Tranche size'])):
    for k in range(0,500):
        if(i==0):
            #print(payoff[1])
            #print(k)
            Price1=Price1+payoff[k]
            #print((1/500)*payoff[k])
        if(i==1):
            Price2=Price2+payoff1[k]
        if(i==2):
            Price3=Price3+payoff2[k]
        if(i==3):
            Price4=Price4+payoff3[k]
        if(i==4):
            Price5=Price5+payoff4[k]

In [34]:
a=0.0254

In [35]:
# Discounting the prices at moody's baseline 1 year rate
PriceofTranche1=Price1*math.exp(-a)
PriceofTranche2=Price2*math.exp(-a)
PriceofTranche3=Price3*math.exp(-a)
PriceofTranche4=Price4*math.exp(-a)
PriceofTranche5=Price5*math.exp(-a)

# Calculating the yield of B/BB and BBB/A rated tranche
Y1 = math.log(Tranche['Tranche size'][0]/PriceofTranche1)*100
Y2 = math.log(Tranche['Tranche size'][1]/PriceofTranche2)*100
Y3 = math.log(Tranche['Tranche size'][2]/PriceofTranche3)*100
Y4 = math.log(Tranche['Tranche size'][3]/PriceofTranche4)*100
Y5 = math.log(Tranche['Tranche size'][4]/PriceofTranche5)*100


Y = [Y1,Y2,Y3,Y4,Y5]
Prices = [PriceofTranche1,PriceofTranche2,PriceofTranche3,PriceofTranche4,PriceofTranche5]
Pricing = ['Tranche 1 no rating','Tranche 2 has no rating','B/BB','BBB/A','Tranche 5 has no rating']

Ratings = pd.DataFrame()
Ratings['Pricing'] = Pricing
Ratings['Prices'] = Prices
Ratings['Yield'] = Y

Ratings.set_index('Pricing')
Ratings.head()

,Pricing,Prices,Yield
0,Tranche 1 no rating,7.333099e+07,107.472566
1,Tranche 2 has no rating,2.604917e+08,12.184880
2,B/BB,1.486963e+08,4.763520
3,BBB/A,4.728231e+08,3.251601
4,Tranche 5 has no rating,2.756020e+10,2.547274
